In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import time

In [3]:
cap= cv2.VideoCapture(0)
while cap.isOpened():
    cap = cv2.VideoCapture(0)
    ret = cap.set(cv2.CAP_PROP_FRAME_HEIGHT,1080)
    ret = cap.set(cv2.CAP_PROP_FRAME_WIDTH,1080)
    ret, frame = cap.read()
    k=cv2.waitKey(1)
    cv2.rectangle(frame,(100,100),(500,500),(255,255,255),2)#Drawing the first rectangle for player 1
    cv2.rectangle(frame,(800,100),(1200,500),(255,255,255),2)#Drawing the second rectangle for player two
    cv2.imshow('frame',frame)
    
    if cv2.waitKey(1) == ord('q'):
        img_name1="firstplayergesture.png"
        firstp=frame[100:500,100:500]
        cv2.imwrite(img_name1,firstp)
        break
cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
img=cv2.imread('firstplayergesture.png') 
hsv1=cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
lower_skin=np.array([0,30,60])
upper_skin=np.array([20,150,255])
kernel=np.ones((3,3),np.uint8)
binary=cv2.inRange(hsv1,lower_skin,upper_skin)
dilution=cv2.dilate(binary,kernel,iterations=1)
erosion=cv2.erode(dilution,kernel,iterations=1)
ret1,the=cv2.threshold(erosion,70,255,cv2.THRESH_BINARY)
cv2.imshow("thresholded_image",the)
contours,hierchy=cv2.findContours(the.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
max_area = -1
for i in range(len(contours)):
            area = cv2.contourArea(contours[i])
            if area>max_area:
                cnt = contours[i]
                max_area = area
                
cnt = cv2.approxPolyDP(cnt,0.01*cv2.arcLength(cnt,True),True)
cv2.drawContours(img, cnt, 0, (0, 255, 0), 3)
cv2.imshow("contoured_image",img)
hull=cv2.convexHull(cnt,returnPoints=False)
print(hull)
defects=cv2.convexityDefects(cnt,hull)
count_defects=0
for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                start = tuple(cnt[s][0])
                end = tuple(cnt[e][0])
                far = tuple(cnt[f][0])
                #If the angle of hand is changed
                a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
                b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
                c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
                angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14
                
                if angle <= 90:
                    count_defects += 1
                    cv2.circle(img, far, 1, [0, 0, 255], -1)
                    cv2.line(img, start, end, [0, 255, 0], 2)
                    
cv2.imshow('image',img)
                    
print(count_defects)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[12]
 [11]
 [ 6]
 [ 3]
 [ 2]
 [ 0]
 [17]
 [15]
 [14]]
1
